In [133]:
import pandas as pd
import datetime as dt

In [31]:
df = pd.read_csv("~/Downloads/COVID-2020.csv")

In [67]:
covid = df[df['tipo_doenca'] == 'COVID']
covid.head(2)

,uf,tipo_doenca,local_obito,faixa_etaria,sexo,total
9,AC,COVID,HOSPITAL,< 9,M,5
31,AL,COVID,HOSPITAL,< 9,F,7


In [68]:
# group the data by state and gender

states = covid.groupby(by=['uf', 'faixa_etaria', 'sexo']).sum()

# Verify if the number of 
assert states['total'].sum() == covid['total'].sum()

In [123]:
# group by local_obito
# Relplace local obito to sumarize the data

possible_places = {
    'AMBULANCIA': 'Ambulancia',
    'DOMICILIO': 'Domicilio',
    'HOSPITAL': 'Hospital',
    'IGNORADO': 'Nao Informado',
    'OUTRO': 'Outros',
    'OUTROS_SERVICOS_SAUDE': 'Outros',
    'POSTO_SAUDE': 'Hospital',
    'SAMU': 'Outros',
    'VIA_PUBLICA': 'Outros',
    
}
# covid['local_obito'] = covid['local_obito'].apply(lambda x: possible_places[x]) 

places = covid.groupby(by=['local_obito']).sum()
# places['total'] = places.loc['total'].sum()
assert places['total'].sum() == covid['total'].sum()

In [85]:
gender = states.groupby('sexo').sum().T
gender['total'] = gender.loc['total'].sum()
assert gender['total'].sum() == covid['total'].sum()

In [ ]:
age = states.groupby('faixa_etaria').sum().T
age['total'] = age.loc['total'].sum()
assert age['total'].sum() == covid['total'].sum()

In [98]:
# Sumarize the data by other diseases
diseases = df.groupby('tipo_doenca').sum().T

In [99]:
diseases.head(1)

tipo_doenca,COVID,INDETERMINADA,INSUFICIENCIA_RESPIRATORIA,OUTRAS,PNEUMONIA,SEPTICEMIA,SRAG
total,103288,5737,61388,414504,114518,97430,11744


In [100]:
states.head(1)

,,,total
uf,faixa_etaria,sexo,
AC,10 - 19,F,3


In [124]:
places.head(1)

,total
local_obito,
Ambulancia,8


In [102]:
gender.head(1)

sexo,F,I,M,total
total,43447,32,59809,103288


In [103]:
age.head(1)

faixa_etaria,10 - 19,20 - 29,30 - 39,40 - 49,50 - 59,60 - 69,70 - 79,80 - 89,90 - 99,< 9,> 100,N/I,total
total,239,1031,3249,6848,13259,22851,26126,21326,7256,433,429,241,103288


In [127]:
# upload all the data to postgresql
datasets = {
    'states': states,
    'age': age,
    'gender': gender,
    'places': places,
    'diseases': diseases
}
postgres_credentials = "postgres_credentials"
from sqlalchemy import engine
engine = engine.create_engine(postgres_credentials)

In [116]:
# upload
for table_name, dataset in datasets.items():
    print('Sending Table %s' % table_name)
#     dataset.to_sql(table_name, engine)

Sending Table age
Sending Table diseases
Sending Table states
Sending Table gender
Sending Table places
